NYC Parking Data Cleaning
==========
(Take 2)
------

# 1. Obtain the data

In [5]:
# NYC LION street Lines
! cd data && curl -O "https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyclion_18a.zip" && unzip nyclion_18a.zip && rm nyclion_18a.zip

# NYC Street Parking Signs
! cd data && curl -O "http://a841-dotweb01.nyc.gov/datafeeds/ParkingReg/Parking_Regulation_Shapefile.zip" && unzip Parking_Regulation_Shapefile.zip && rm Parking_Regulation_Shapefile.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21.8M  100 21.8M    0     0  1629k      0  0:00:13  0:00:13 --:--:-- 1926k
Archive:  Parking_Regulation_Shapefile.zip
  inflating: Parking_Regulation_Shapefile/Parking Regulation Metadata.pdf  
 extracting: Parking_Regulation_Shapefile/Parking_Regulation_Shapefile.CPG  
  inflating: Parking_Regulation_Shapefile/Parking_Regulation_Shapefile.dbf  
  inflating: Parking_Regulation_Shapefile/Parking_Regulation_Shapefile.prj  
  inflating: Parking_Regulation_Shapefile/Parking_Regulation_Shapefile.sbn  
  inflating: Parking_Regulation_Shapefile/Parking_Regulation_Shapefile.sbx  
  inflating: Parking_Regulation_Shapefile/Parking_Regulation_Shapefile.shp  
  inflating: Parking_Regulation_Shapefile/Parking_Regulation_Shapefile.shp.xml  
  inflating: Parking_Regulation_Shapefile/Parking_Regulation_Shapefile.shx  


In [127]:
! pip install -U fiona dateutil
! brew upgrade gdal

Requirement already up-to-date: fiona in /anaconda3/lib/python3.6/site-packages
Requirement already up-to-date: six in /anaconda3/lib/python3.6/site-packages (from fiona)
Requirement already up-to-date: click-plugins in /anaconda3/lib/python3.6/site-packages (from fiona)
Requirement already up-to-date: cligj in /anaconda3/lib/python3.6/site-packages (from fiona)
Requirement already up-to-date: munch in /anaconda3/lib/python3.6/site-packages (from fiona)
Requirement already up-to-date: click>=3.0 in /anaconda3/lib/python3.6/site-packages (from click-plugins->fiona)
You are using pip version 9.0.2, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Updating Homebrew...
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core, caskroom/cask).
No changes to formulae.

Error: gdal 2.2.4 already installed
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
You 

In [64]:
import pandas as pd
import geopandas as gpd
import re
from dateutil.rrule import rrule, WEEKLY
from datetime import datetime

In [2]:
signs = gpd.read_file('data/Parking_Regulation_Shapefile/Parking_Regulation_Shapefile.shp')

In [3]:
# Convert LION Geodatabase to a GeoJSON file because Geopandas has FEELINGS! about importing geodatabase layers
! ogr2ogr -f "GeoJSON" data/lion/lion.geojson data/lion/lion.gdb lion

In [ ]:
streets = gpd.read_file('data/lion/lion.geojson')

In [144]:
signs.sample(10)

,SG_KEY_BOR,SG_ORDER_N,SG_SEQNO_N,SG_MUTCD_C,SR_DIST,SG_SIGN_FC,SG_ARROW_D,X,Y,SIGNDESC,SIGNDESC1,geometry
167660,M,S-350195,28,PS-2G,NULL,NULL,NULL,995847,232059,NO PARKING ANYTIME <-> (SUPERSEDES SP-854C),NO PARKING ANYTIME <->,POINT (-73.95811230749092 40.80362489758593)
293098,K,S-962715,5,SP-360C,NULL,NULL,NULL,998742,179887,NO PARKING (SANITATION BROOM SYMBOL) 11AM TO 1...,NO PARKING (SANITATION BROOM SYMBOL) 11AM TO 1...,POINT (-73.94776773702527 40.66042132013271)
356572,S,S-912980,4,R7-40,NULL,NULL,NULL,918443,126584,NO PARKING ANYTIME(SUPERSEDED BY R7-40R) DON'T...,NO PARKING ANYTIME,POINT (-74.23667085201899 40.51388305712407)
371062,Q,S-499851,3,PS-37B,NULL,NULL,NULL,1017070,215526,NO PARKING (SANITATION BROOM SYMBOL) MONDAY 11...,NO PARKING (SANITATION BROOM SYMBOL) MONDAY 11...,POINT (-73.88153422693605 40.75819288377527)
117350,K,S-161514,3,SP-379C,NULL,NULL,NULL,996650,155052,NO PARKING (SANITATION BROOM SYMBOL) 11:30AM T...,NO PARKING (SANITATION BROOM SYMBOL) 11:30AM T...,POINT (-73.95535357900746 40.59225763637203)
220620,M,S-01255104,8,R7-40R,NULL,NULL,NULL,995294,218145,NO PARKING ANYTIME <----> (SUPERSEDES R7-40 DO...,NO PARKING ANYTIME <---->,POINT (-73.96013265254977 40.76543538012491)
369100,Q,P-340631,12,PS-145C,NULL,NULL,NULL,996649,209390,4 HMP 8AM-8PM EXCEPT SUNDAY <->,4 HMP 8AM-8PM EXCEPT SUNDAY <->,POINT (-73.9552572510672 40.74140332822775)
197371,M,P-452745,13,PS-55CA,NULL,NULL,E,1000907,249458,1 HOUR METERED PARKING 9AM-7PM EXCEPT SUNDAY -...,1 HOUR METERED PARKING 9AM-7PM EXCEPT SUNDAY -->,POINT (-73.9397920576526 40.85137209493217)
133166,K,S-297205,6,SP-413C,NULL,NULL,NULL,1000079,188262,NO PARKING (SANITATION BROOM SYMBOL) 9-10:30AM...,NO PARKING (SANITATION BROOM SYMBOL) 9-10:30AM...,POINT (-73.94292910581943 40.68340652866513)
296031,Q,S-201782,5,R7-61A,NULL,NULL,E,1012173,203836,NO STANDING(SINGLE ARROW)HANDICAP BUS(SYMBOL)W...,NO STANDING,POINT (-73.89925897008968 40.72612358396288)


In [3]:
aspsigns = signs[signs.SIGNDESC1.str.contains('\(SANITATION BROOM SYMBOL\)')]

In [4]:
aspsigns.SIGNDESC1.value_counts()

NO PARKING (SANITATION BROOM SYMBOL) 11:30AM TO 1PM MON & THURS <---->                                                    3955
NO PARKING (SANITATION BROOM SYMBOL) 11:30AM TO 1PM TUES & FRI <---->                                                     3928
NO PARKING (SANITATION BROOM SYMBOL) TUESDAY FRIDAY 11:30AM-1PM <->                                                       3638
NO PARKING (SANITATION BROOM SYMBOL) MONDAY THURSDAY 11:30AM-1PM <->                                                      3567
NO PARKING (SANITATION BROOM SYMBOL) 9:30-11AM MON & THURS <--->                                                          3011
NO PARKING (SANITATION BROOM SYMBOL) 11:30AM TO 1PM TUES <---->                                                           2978
NO PARKING (SANITATION BROOM SYMBOL) 11AM TO 12:30PM TUES & FRI <---->                                                    2914
NO PARKING (SANITATION BROOM SYMBOL) 11:30AM TO 1PM MON <--->                                                  

In [12]:
aspsigns[aspsigns.SIGNDESC1.str.contains("EXCEPT")].SIGNDESC1.value_counts()

NO PARKING (SANITATION BROOM SYMBOL) 7:30AM-8AM EXCEPT SUNDAY <->                                                     2615
NO PARKING (SANITATION BROOM SYMBOL) 8AM-8:30AM EXCEPT SUNDAY <->                                                     2426
NO PARKING (SANITATION BROOM SYMBOL) 8:30AM-9AM EXCEPT SUNDAY <->                                                     1502
NO PARKING (SANITATION BROOM SYMBOL) 7:30AM-8AM EXCEPT SUNDAY -->                                                      988
NO PARKING (SANITATION BROOM SYMBOL) 8AM-8:30AM EXCEPT SUNDAY -->                                                      929
NO PARKING (SANITATION BROOM SYMBOL) 8:30AM-9AM EXCEPT SUNDAY -->                                                      610
NO PARKING (SANITATION BROOM SYMBOL) 7AM-7:30AM EXCEPT SUNDAY <->                                                      299
NO PARKING (SANITATION BROOM SYMBOL) 8AM-9AM EXCEPT SUNDAY <->                                                         287
NO PARKING (SANI

## What is the data model for a parking restriction?

* Street Segment
* Day(s) of week
* Time of Restriction (on those days)


In [120]:
TIMERE = re.compile('(\d+:?\d*[A|P]?M?|NOON|MIDNIGHT)(?:-| TO | | TO-|)(\d+:?\d*[A|P]?M?|NOON|MIDNIGHT)')
MIDDAY = re.compile('([A|P]M)')
EXCEPTDAYS = re.compile('EXCEPT (MONDAY|TUESDAY|WEDNESDAY|THURSDAY|FRIDAY|SATURDAY|SUNDAY|MON|TUES|WED|THURS|FRI|SAT|SUN)+')
RANGEDAYS = re.compile('(MONDAY|TUESDAY|WEDNESDAY|THURSDAY|FRIDAY|SATURDAY|SUNDAY|MON|TUES|WED|THURS|FRI|SAT|SUN)-(MONDAY|TUESDAY|WEDNESDAY|THURSDAY|FRIDAY|SATURDAY|SUNDAY|MON|TUES|WED|THURS|FRI|SAT|SUN)')
ALLDAYS = re.compile('(MONDAY|TUESDAY|WEDNESDAY|THURSDAY|FRIDAY|SATURDAY|SUNDAY|MON|TUES|WED|THURS|FRI|SAT|SUN)')

In [7]:
TIMERE.findall('NO PARKING (SANITATION BROOM SYMBOL) 10:30AM TO NOON MON & THURS W/SINGLE ARROW')

[('10:30AM', 'NOON')]

In [21]:
smp = aspsigns.sample(1000)

In [22]:


for ix, row in smp.iterrows():

    try:
        timeFrom, timeTo = TIMERE.findall(row['SIGNDESC1'])[0]
    except:
        print(row)
        raise
    if timeFrom == "NOON": timeFrom = "12:00PM"
    if timeFrom == "MIDNIGHT": timeFrom = "12:00AM"
    if timeTo == "NOON": timeTo = "12:00PM"
    if timeTo == "MIDNIGHT": timeTo = "12:00AM"
    if not MIDDAY.search(timeFrom) and MIDDAY.search(timeTo):
        timeFrom += (MIDDAY.findall(timeTo)[0])
        

In [8]:
def timeRules(signdesc):
    try:
        timeFrom, timeTo = TIMERE.findall(signdesc)[0]
    except:
        print(signdesc)
        raise
    if timeFrom == "NOON":
        timeFrom = "12:00PM"
    if timeFrom == "MIDNIGHT":
        timeFrom = "12:00AM"
    if timeTo == "NOON":
        timeTo = "12:00PM"
    if timeTo == "MIDNIGHT":
        timeTo = "12:00AM"
    if not MIDDAY.search(timeFrom) and MIDDAY.search(timeTo):
        timeFrom += (MIDDAY.findall(timeTo)[0])
    
    return timeFrom, timeTo

In [112]:
NORMALIZEDAYS = {
    'SUNDAY': 'SUNDAY',
    'SUN': 'SUNDAY',
    'MONDAY': 'MONDAY',
    'MON': 'MONDAY',
    'TUESDAY': 'TUESDAY',
    'TUE': 'TUESDAY',
    'TUES': 'TUESDAY',
    'WEDNESDAY': 'WEDNESDAY',
    'WED': 'WEDNESDAY',
    'WEDS': 'WEDNESDAY',
    'THURSDAY': 'THURSDAY',
    'THU': 'THURSDAY',
    'THUR': 'THURSDAY',
    'THURS': 'THURSDAY',
    'FRIDAY': 'FRIDAY',
    'FRI': 'FRIDAY',
    'SATURDAY': 'SATURDAY',
    'SAT': 'SATURDAY'
}
def dayRules(signdesc):
    exceptday = EXCEPTDAYS.findall(signdesc)
    rangeday = RANGEDAYS.findall(signdesc)
    alldays = ALLDAYS.findall(signdesc)

    dayslist = ['SUNDAY', 'MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY']
    if exceptday:
        dayslist = ['SUNDAY', 'MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY']
        
        for d in exceptday:
            day = NORMALIZEDAYS[d]
            dayslist.remove(day)
        return dayslist
    elif rangeday:
        #print(rangeday)
        fromDay, toDay = rangeday[0]
        fromIx = dayslist.index(fromDay)
        toIx = dayslist.index(toDay)
        #print(fromIx,toIx)
        return dayslist[fromIx:toIx+1]
    else:
        return alldays
        
        
    
        

In [143]:
for i in vc.index:
    days = dayRules(i)
    try:
        fromTime, toTime = timeRules(i)
    except:
        print(i)
        pass


NO PARKING (SANITATION BROOM SYMBOL) <->
NO PARKING (SANITATION BROOM SYMBOL) <->


# Side Quest: Lets do topic modeling on these sign texts!

Lets look at the unique text of each sign 

In [158]:
count_signs = signs.groupby(['SG_MUTCD_C', 'SIGNDESC1']).size().reset_index(name="Count").sort_values(by="Count", ascending=False)

In [170]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        
documents = list(count_signs['SIGNDESC1'])

no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

no_topics = 50

# Run NMF
# nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

no_top_words = 5
#display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
available description www logo visit
Topic 1:
taxi stand limit symbol hour
Topic 2:
driveway clear broom symbol sanitation
Topic 3:
tunnel bridge tbta august crosswalk
Topic 4:
large lottery stand panel permit
Topic 5:
star symbol avo dept nyc
Topic 6:
arrow sign symbol clock right
Topic 7:
truck symbol loading 6pm friday
Topic 8:
curve president chevron studios hazard
Topic 9:
muni meter pay vehicle marshals
Topic 10:
7pm parking hour 9am metered
Topic 11:
hour 7pm parking 30am metered
Topic 12:
market farmers 30 nov sun
Topic 13:
parking symbol sanitation broom 30am
Topic 14:
information dot parking emergency telecommunications
Topic 15:
tour park layover slippery saturday
Topic 16:
7pm standing monday friday 10am
Topic 17:
diamond red delineator pictorial inch
Topic 18:
mon fri parking standing 8am
Topic 19:
broom 30am symbol parking sanitation
Topic 20:
11am 30pm 12 symbol crossing
Topic 21:
office general mayor justice medical
Topic 22:
city day times bus xyy
Topic 23:
co